In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import PIL
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import random
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import save
from numpy import load

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import Input, Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten

In [2]:
X = load('D:/Desktop/desktop data/cat dog/data128(3).npy')
y = load('D:/Desktop/desktop data/cat dog/y128(3)(all).npy')

In [3]:
y = y.reshape(45091,1)
y = y.flatten()

In [4]:
X.shape, y.shape

((45091, 128, 128, 3), (45091,))

In [5]:
x_train = X[:36000]
y_train = y[:36000]
x_dev = X[36000:40500]
y_dev = y[36000:40500]
x_test = X[40500:]
y_test = y[40500:]

In [6]:
x_train.shape,y_train.shape,x_dev.shape,y_dev.shape,x_test.shape,y_test.shape

((36000, 128, 128, 3),
 (36000,),
 (4500, 128, 128, 3),
 (4500,),
 (4591, 128, 128, 3),
 (4591,))

In [7]:
IMG_SHAPE = (128, 128, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

80142336/80134624 [==============================] - 375s 5us/step


In [8]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [7]:
x = base_model.get_layer('block4_pool').output
prediction = tf.keras.layers.Flatten()(x)

In [8]:
model = Model(inputs=base_model.input, outputs=prediction)

In [9]:
%%time
block4_pool_features = model.predict(x_train)
block4_pool_features_dev = model.predict(x_dev)
block4_pool_features_test = model.predict(x_test)

Wall time: 2min 52s


In [10]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [ ]:
block4_pool_features.shape

In [ ]:
block4_pool_features_test.shape

In [ ]:
%%time
svm = LinearSVC()
svm.fit(block4_pool_features, y_train)

In [ ]:
preds = svm.predict(block4_pool_features_dev)

In [ ]:
preds_test = svm.predict(block4_pool_features_test)

In [ ]:
accuracy_score(y_dev, preds), accuracy_score(y_test, preds_test)

In [11]:
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression

In [ ]:
%%time
knn = KNeighborsClassifier()
knn.fit(block4_pool_features, y_train)

In [ ]:
preds = knn.predict(block4_pool_features_dev)
preds_test = knn.predict(block4_pool_features_test)

In [ ]:
accuracy_score(y_dev, preds), accuracy_score(y_test, preds_test)

In [ ]:
%%time
lr = LinearRegression()
lr.fit(block4_pool_features, y_train)

In [ ]:
preds = lr.predict(block4_pool_features_dev)
preds_test = lr.predict(block4_pool_features_test)

In [ ]:
accuracy_score(y_dev, preds), accuracy_score(y_test, preds_test)